In [ ]:
def send_pi_messages():
    piq = get_pi_queue()
    netq = get_network_transfer_queue()
    while True:
        cfg = piq.peek()
        if cfg.running:
            set_pi_running_bool(True)
            http = HTTPSession(cfg.username, cfg.password, verify_ssl=True)
            prefix = "%s,%s,%s,"%(cfg.asset_name, cfg.template_name, cfg.token_name)
            var_names = get_global_names_to_send()
            run_message_pump(netq, var_names, prefix, url, http, cfg.wait)
            http.close()
        else:
            wait_for_a_bit_like_500ms_or_cfg_change_notice()
        stop = get_shutdown_boolean()
        if stop:
            break
            
        
def run_message_pump(netq, var_names, prefix, url, http, wait):
    while True:
        pbs_globals = netq.peek()
        msg = format_pi_message(pbs_globals, var_names, prefix)
        send_put_message(http, msg, url)
        stop = config_has_changed() or get_shutdown_boolean()
        if stop:
            break
        wait_until_next_sane_time_interval(wait)